# Hands-on: Build Multi-agent AI system using watsonx.ai & CrewAI

# Overview
This hands-on notebook demonstrates the concept of multiagent systems in AI, where multiple autonomous agents collaborate to achieve complex goals. The notebook employs the crewai library to create and manage a team of agents, each specialized in a specific task, using IBM watsonx LLM and other tools to execute their roles.


# Definition of Multi-agent System
A multiagent system is a system where multiple intelligent agents interact or work together to solve problems that are beyond the capabilities of a single agent. These agents can operate independently or collaboratively, each bringing their own expertise and capabilities to the table. In this notebook, the multiagent system is used to simulate a research and writing scenario involving two specialized agents.


# Scenario
The scenario involves two agents working together to produce a detailed output on the topic of quantum computing:

`Researcher Agent`: A senior AI researcher tasked with finding promising research in the field of quantum computing. This agent uses IBM watsonx LLM and a search tool to gather information and summarize the findings.

`Writer Agent`: A senior speech writer responsible for crafting an engaging keynote speech based on the research provided by the Researcher Agent. This agent leverages the IBM watsonx LLM to structure and write the speech.

The Crew class orchestrates the workflow, coordinating the tasks and ensuring that each agent's output feeds into the next step of the process. The agents work autonomously within their defined roles, contributing to the overall objective of producing a well-researched and well-written keynote speech on quantum computing.


In [1]:
# Install library
!pip install crewai==0.22.5
!pip install crewai-tools==0.0.15
!pip install langchain-ibm==0.1.3

  Using cached langchain_openai-0.0.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached tiktoken-0.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)


Using cached langchain_openai-0.0.5-py3-none-any.whl (29 kB)
Using cached tiktoken-0.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.7.0
    Uninstalling tiktoken-0.7.0:
      Successfully uninstalled tiktoken-0.7.0
  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 0.1.7
    Uninstalling langchain-openai-0.1.7:
      Successfully uninstalled langchain-openai-0.1.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.113 requires langchain-openai<0.2.0,>=0.1.7, but you have langchain-openai 0.0.5 which is incompatible.
embedchain 0.1.113 requires tiktoken<0.8.0,>=0.7.0, but you have tiktoken 0.5.2 which is incompatible.


  Using cached langchain_openai-0.1.22-py3-none-any.whl.metadata (2.6 kB)
  Using cached tiktoken-0.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)


INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_openai-0.1.20-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_openai-0.1.19-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_openai-0.1.17-py3-none-any.whl.metadata (2.5 kB)
  Using cached langchain_openai-0.1.16-py3-none-any.whl.metadata (2.5 kB)
  Using cached langchain_openai-0.1.15-py3-none-any.whl.metadata (2.5 kB)
  Using cached langchain_openai-0.1.14-py3-none-any.whl.metadata (2.5 kB)
  Using cached langchain_openai-0.1.13-py3-none-any.whl.metadata (2.5 kB)
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_openai-0.1.12-py3-none-any.whl.metadata (2.5 kB)
  Using cached langchain_openai-0.1.11-py3-none-any.whl.metadata (2.5 kB)
  Using cached langcha

Using cached langchain_openai-0.1.7-py3-none-any.whl (34 kB)
Using cached tiktoken-0.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.5.2
    Uninstalling tiktoken-0.5.2:
      Successfully uninstalled tiktoken-0.5.2
  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 0.0.5
    Uninstalling langchain-openai-0.0.5:
      Successfully uninstalled langchain-openai-0.0.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crewai 0.22.5 requires langchain-openai<0.0.6,>=0.0.5, but you have langchain-openai 0.1.7 which is incompatible.


In [2]:
# Import libraries
import os
import warnings

from crewai import Crew, Task, Agent
from crewai_tools import SerperDevTool
from langchain_ibm import WatsonxLLM

warnings.filterwarnings("ignore")

Get your Serper API Key here: https://serper.dev/api-key

In [3]:
os.environ["WATSONX_APIKEY"] = "<YOUR IBM CLOUD API KEY HERE>"
os.environ["SERPER_API_KEY"] = "<YOUR SERPER API KEY HERE>"

## Primary LLM: 
- The primary LLM (llm) is used by both agents to generate text based on the tasks assigned to them. This involves producing summaries, writing detailed speeches, or other natural language generation tasks.
- `Researcher Agent`: The researcher agent uses the primary LLM to generate summaries of the research findings. For instance, when the agent is tasked with finding promising AI research, it processes the information using the LLM to create bullet-point summaries.
- `Writer Agent`: The writer agent uses the primary LLM to draft the keynote speech. The LLM helps the agent structure the speech, write engaging content, and ensure coherence and flow in the text.

## Function Calling LLM
- The function_calling_llm is used when the agent needs to interact with external tools or perform specific functions beyond generating text. In this notebook:
- `Researcher Agent`: The researcher agent uses the function_calling_llm when it needs to search the internet for research papers. The function calling LLM allows the agent to perform a search query using the SerperDevTool and process the results to extract relevant information.
- `Tool Integration`: Function calling enables the agent to delegate tasks to the search tool or other integrated functionalities, executing commands or retrieving specific data that the agent can then process using the primary LLM.

In [4]:
# Parameters
parameters = {"decoding_method": "greedy", 
              "max_new_tokens": 500}

# Create the first LLM
llm = WatsonxLLM(
    model_id="meta-llama/llama-3-70b-instruct",
    url="https://us-south.ml.cloud.ibm.com",
    params=parameters,
    project_id="YOUR WATSONX.AI PROJECT ID HERE>",
)

# Create the function calling llm
function_calling_llm = WatsonxLLM(
    model_id="ibm-mistralai/merlinite-7b",
    url="https://us-south.ml.cloud.ibm.com",
    params=parameters,
    project_id="3YOUR WATSONX.AI PROJECT ID HERE>",
)

# Tools
search = SerperDevTool()

# Create the agent
researcher = Agent(
    llm=llm,
    function_calling_llm=function_calling_llm,
    role="Senior AI Researcher",
    goal="Find promising research in the field of quantum computing.",
    backstory="You are a veteran quantum computing researcher with a background in modern physics.",
    allow_delegation=False,
    tools=[search],
    verbose=1,
)

# Create a task
task1 = Task(
    description="Search the internet and find 5 examples of promising AI research.",
    expected_output="A detailed bullet point summary on each of the topics. Each bullet point should cover the topic, background and why the innovation is useful.",
    output_file="task1output.txt",
    agent=researcher,
)

# Create the second agent
writer = Agent(
    llm=llm,
    role="Senior Speech Writer",
    goal="Write engaging and witty keynote speeches from provided research.",
    backstory="You are a veteran quantum computing writer with a background in modern physics.",
    allow_delegation=False,
    verbose=1,
)

# Create a task
task2 = Task(
    description="Write an engaging keynote speech on quantum computing.",
    expected_output="A detailed keynote speech with an intro, body and conclusion.",
    output_file="task2output.txt",
    agent=writer,
)

# Put all together with the crew
crew = Crew(agents=[researcher, writer], tasks=[task1, task2], verbose=1)
print(crew.kickoff())

 [DEBUG]: == Working Agent: Senior AI Researcher


> Entering new CrewAgentExecutor chain...
I need to find promising AI research in the field of quantum computing.

Action: 
Search the internet

Action Input: 
{"search_query": "promising AI research in quantum computing"}

Observation 


Search results: Title: The AI–quantum computing mash-up: will it revolutionize science?
Link: https://www.nature.com/articles/d41586-023-04007-0
Snippet: Scientists are exploring the potential of quantum machine learning. But whether there are useful applications for the fusion of artificial ...
---
Title: Quantum Computing and AI: A Perfect Match? - InformationWeek
Link: https://www.informationweek.com/machine-learning-ai/quantum-computing-and-ai-a-perfect-match-
Snippet: Quantum AI is a burgeoning computer science sector, dedicated to exploring the potential synergy that exists between quantum computing and AI, ...
---
Title: Supercharging AI with Quantum Computing: A Look into the Future
Link: http

Here is the final answer:

Thought: I now know the final answer

Final Answer: 

Here are 5 examples of promising AI research in the field of quantum computing:

• **Quantum Machine Learning**: This field explores how to devise and implement quantum software that could enable machine learning that is faster, more efficient, and more accurate than classical machine learning. Quantum machine learning uses algorithms run on quantum devices, such as quantum computers, to supplement, expedite, or support the work of classical machine learning algorithms. This has the potential to revolutionize fields such as image and speech recognition, natural language processing, and predictive analytics.

• **Quantum AI Applications**: Quantum AI applications aim to apply artificial intelligence, statistics, and quantum computing to strategic industries, pursuing and protecting the good of humanity. This includes using quantum computing for the rapid training of machine learning models and creating opti